## PySpark Demo - (In Jupyter Notebook)

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

rdd = sc.parallelize([1,2,3,4])
rdd.collect()

[1, 2, 3, 4]

## ## PySpark Demo - (In Zepplin)

In [ ]:
%pyspark

rdd = sc.parallelize([1,2,3,4])
rdd.collect()

## Iterative Programming

In [4]:
# loading data
a = [1,2,3,4,5,6,7,8]

# get even number
b = []
for ele in a:
    if ele % 2 == 0: 
        b.append(ele)
        
# get sum of even number
c = sum(b)
c

20

## Functional Programming

In [9]:
import numpy as np
a = np.array([1,2,3,4,5,6,7,8])

def f(ele):
    return ele % 2 == 0

sum(a[f(a)])


'/home/jovyan'